In [1]:
import pandas as pd

import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

from sklearn.metrics import classification_report
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.pipeline import Pipeline, FeatureUnion
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.externals import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import precision_score
from rdkit.Chem import rdMolDescriptors, MolFromSmiles, rdmolfiles, rdmolops

Using TensorFlow backend.
C:\Users\Khallef\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Khallef\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Khallef\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Khallef\Anaconda3\lib\site-packages\tensorflow\python\framework\d

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Khallef\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Khallef\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Khallef\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predi

In [2]:
def fingerprint_features(smile_string, radius=2, size=2048):
    mol = MolFromSmiles(smile_string)
    new_order = rdmolfiles.CanonicalRankAtoms(mol)
    mol = rdmolops.RenumberAtoms(mol, new_order)
    return rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius,
                                                          nBits=size,
                                                          useChirality=True,
                                                          useBondTypes=True,
                                                          useFeatures=False
                                                          )

In [3]:
fingerprint_features('Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C')

# Model 1

In [4]:
class ModelOne:
    
    def __init__(self, csv_filepath):
        self.csv_filepath = csv_filepath
        
    
    def data_prepocessing(self):
        """
        Implement load_data

        Arguements:
        csv_filepath -- the path directory of the csv_filepath in the workspace

        Returnes:
        X, y

        """

        df = pd.read_csv(self.csv_filepath)

        df['extracted features'] = df.apply(lambda row : fingerprint_features(row['smiles']), axis = 1)

        df['ExplicitBitVect_to_array'] = df.apply(lambda row : np.frombuffer(row['extracted features'].ToBitString().encode(), 'u1') - ord('0'), axis = 1)

        X = df['ExplicitBitVect_to_array']

        y = df['P1']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

        X_train = np.stack(X_train.values)

        X_test = np.stack(X_test.values)

        return X_train, X_test, y_train, y_test
    
    def build_model(self):
        """
        Implement build_model

        Arguements:

        Returnes:
        the builded model
        """

        model = models.Sequential()

        model.add(layers.Dense(512, activation='relu', input_shape=(2048*1,)))

        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model
    
    def evaluate_model(self, model, X_test, Y_test):
        """
        Implement evaluate model

        Retuerns:
        report the f1 score, precision and recall
        """

        # predict on test data
        # Y_pred = model.predict(X_test)

        test_loss, test_acc = model.evaluate(X_test, Y_test)
        print('test_acc:', test_acc)
        
    

In [5]:
modelone = ModelOne('dataset_single.csv')

print('Loading data...\n')

X_train, X_test, y_train, y_test = modelone.data_prepocessing()


print('Building model...')

model_1 = modelone.build_model()

print('Training model...')

model_1.fit(X_train, y_train, epochs=10, batch_size=128)

print('Evaluating model...')
modelone.evaluate_model(model_1, X_test, y_test)


Loading data...

Building model...
Instructions for updating:
Colocations handled automatically by placer.
Training model...
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
3349/3349 [==============================] - 1s 387us/step - loss: 0.4994 - acc: 0.7952
Epoch 2/10
3349/3349 [==============================] - 1s 279us/step - loss: 0.3786 - acc: 0.8304
Epoch 3/10
3349/3349 [==============================] - 1s 279us/step - loss: 0.3138 - acc: 0.8549
Epoch 4/10
3349/3349 [==============================] - 1s 266us/step - loss: 0.2494 - acc: 0.8964
Epoch 5/10
3349/3349 [==============================] - 1s 266us/step - loss: 0.1903 - acc: 0.9301
Epoch 6/10
3349/3349 [==============================] - 1s 264us/step - loss: 0.1381 - acc: 0.9654 0s - loss: 0.1422 - acc: 0.
Epoch 7/10
3349/3349 [==============================] - 1s 259us/step - loss: 0.0961 - acc: 0.9836
Epoch 8/10
3349/3349 [==============================] - 1s 275us/step - loss: 0.0656 - acc: 0.9934
Epoch 9

In [6]:
preds = model_1.predict(X_test)
classe_one = preds > 0.5
classe_two = y_test > 0
print(classification_report(classe_two, classe_one))

              precision    recall  f1-score   support

       False       0.38      0.16      0.22       290
        True       0.84      0.95      0.89      1360

   micro avg       0.81      0.81      0.81      1650
   macro avg       0.61      0.55      0.55      1650
weighted avg       0.76      0.81      0.77      1650



In [433]:
# fingerprint_features('Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C')
featur = fingerprint_features('Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C')
featur = np.frombuffer(featur.ToBitString().encode(), 'u1') - ord('0')



In [441]:
f1 = featur.reshape((1, 2048))

In [446]:
model_1.predict(f1)


array([[0.9890758]], dtype=float32)

In [447]:
df

,P1,mol_id,smiles
0,1,CID2999678,Cc1cccc(N2CCN(C(=O)C34CC5CC(CC(C5)C3)C4)CC2)c1C
1,0,CID2999679,Cn1ccnc1SCC(=O)Nc1ccc(Oc2ccccc2)cc1
2,1,CID2999672,COc1cc2c(cc1NC(=O)CN1C(=O)NC3(CCc4ccccc43)C1=O...
3,0,CID5390002,O=C1/C(=C/NC2CCS(=O)(=O)C2)c2ccccc2C(=O)N1c1cc...
4,1,CID2999670,NC(=O)NC(Cc1ccccc1)C(=O)O
...,...,...,...
4994,1,CID3239880,CC1CCC(NC(=O)CN2CCCN(Cc3ccc(F)cc3Cl)C2=O)CC1
4995,1,CID1491423,Cc1cccc(-n2cnc(C(=O)Nc3cccnc3)c2)n1
4996,1,CID3239886,COc1ccc(CCNC(=O)c2noc3c2CCCC3)cc1OC
4997,1,CID3239885,COCc1ccc2oc(C(=O)O)cc2c1


# Madel 2

In [7]:
class ModelTwo:
    
    def __init__(self, csv_filepath):
        self.csv_filepath = csv_filepath
        
    def data_prepocessing(self):
        """
        Implement load_data

        Arguements:
        csv_filepath -- the path directory of the csv_filepath in the workspace

        Returnes:
        X_train, X_test, y_train, y_test

        """

        df = pd.read_csv(self.csv_filepath)

        X = df['smiles'].values
        y = df['P1'].values
        
        vectorizer = CountVectorizer()
        
        vectorizer.fit(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)
        
        X_train = vectorizer.transform(X_train)

        X_test  = vectorizer.transform(X_test)

        return X_train, X_test, y_train, y_test
    
    
    def build_model(self, X_train):
        """
        Implement build_model

        Arguements:

        Returnes:
        the builded model
        """

        input_dim = X_train.shape[1]  # Number of features

        model = Sequential()

        model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))

        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model
    
    def evaluate_model(self, model, X_train, X_test, y_train, y_test):
        """
        Implement evaluate model

        Retuerns:
        report the f1 score, precision and recall
        """

        # predict on test data
        # Y_pred = model.predict(X_test)

        loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
        print("Training Accuracy: {:.4f}".format(accuracy))
        loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
        print("Testing Accuracy:  {:.4f}".format(accuracy))
    
    

In [8]:
modeltwo = ModelTwo('dataset_single.csv')

print('Loading data...\n')

X_train, X_test, y_train, y_test = modeltwo.data_prepocessing()


print('Building model...')

model_2 = modeltwo.build_model(X_train)

print('Training model...')

model_2.fit(X_train, y_train, epochs=10, verbose=False, validation_data=(X_test, y_test), batch_size=10)

print('Evaluating model...')
modeltwo.evaluate_model(model_2, X_train, X_test, y_train, y_test)

Loading data...

Building model...
Training model...
Evaluating model...
Training Accuracy: 0.9370
Testing Accuracy:  0.7728


In [9]:
preds = model_2.predict(X_test)
classe_one = preds > 0.5
classe_two = y_test > 0
print(classification_report(classe_two, classe_one))

              precision    recall  f1-score   support

       False       0.25      0.13      0.17       225
        True       0.83      0.91      0.87      1025

   micro avg       0.77      0.77      0.77      1250
   macro avg       0.54      0.52      0.52      1250
weighted avg       0.72      0.77      0.74      1250



# Model 3

In [10]:
class ModelThree:
    
    def __init__(self, csv_filepath):
        self.csv_filepath = csv_filepath
    
    def data_prepocessing(self):
        """
        Implement load_data

        Arguements:
        csv_filepath -- the path directory of the csv_filepath in the workspace

        Returnes:
        X_train, X_test, y_train, y_test

        """

        df = pd.read_csv(self.csv_filepath)

        X = df['smiles'].values

        y = df[["P2", "P1", "P3", "P4", "P5","P6", "P7", "P8", "P9"]]

        y = y.to_numpy()
        
        vectorizer = CountVectorizer()

        vectorizer.fit(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1000)      

        X_train = vectorizer.transform(X_train)

        X_test  = vectorizer.transform(X_test)

        return X_train, X_test, y_train, y_test
    
    
    def build_model(self):
        """
        Implement build_model

        Arguements:

        Returnes:
        the builded model
        """

        pipeline = Pipeline([

            ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

        parameters = {

        }

        cv = GridSearchCV(pipeline, param_grid=parameters)

        return cv
    
    
    def evaluate_model(self, model, X_test, y_test):
        """
        Implement evaluate model

        Retuerns:
        report the f1 score, precision and recall for each output category of the dataset
        """

        # predict on test data
        y_pred = model.predict(X_test)

        print('Classification report ')

        print(classification_report(y_test, y_pred))
        
    

In [11]:
modelthree = ModelThree('dataset_multi.csv')

print('Loading data...\n')

X_train, X_test, y_train, y_test = modelthree.data_prepocessing()


print('Building model...')

model_3 = modelthree.build_model()

print('Training model...')

model_3.fit(X_train, y_train)

print('Evaluating model...')
modelthree.evaluate_model(model_3, X_test, y_test)


Loading data...

Building model...
Training model...


C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\model_selection\_split

C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To sile

C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check 

C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your 

...

C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check 

C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `

C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\tree\tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check 

Evaluating model...
Classification report 
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      1034
           1       0.83      0.95      0.88      1025
           2       0.85      0.95      0.90      1060
           3       0.83      0.93      0.88      1042
           4       0.84      0.94      0.89      1037
           5       0.84      0.95      0.89      1050
           6       0.84      0.94      0.88      1046
           7       0.84      0.95      0.89      1045
           8       0.85      0.95      0.90      1060

   micro avg       0.84      0.95      0.89      9399
   macro avg       0.84      0.95      0.89      9399
weighted avg       0.84      0.95      0.89      9399
 samples avg       0.84      0.94      0.88      9399



C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link 

In [12]:
print('Evaluating model...')
modelthree.evaluate_model(model_3, X_test, y_test)

Evaluating model...
Classification report 
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      1034
           1       0.83      0.95      0.88      1025
           2       0.85      0.95      0.90      1060
           3       0.83      0.93      0.88      1042
           4       0.84      0.94      0.89      1037
           5       0.84      0.95      0.89      1050
           6       0.84      0.94      0.88      1046
           7       0.84      0.95      0.89      1045
           8       0.85      0.95      0.90      1060

   micro avg       0.84      0.95      0.89      9399
   macro avg       0.84      0.95      0.89      9399
weighted avg       0.84      0.95      0.89      9399
 samples avg       0.84      0.94      0.88      9399



C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\Khallef\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link 